Using Amazon Redshift ML, create a model referencing the SageMaker JumpStart LLM endpoint

In [0]:
CREATE MODEL falcon_7b_instruct_llm_model
FUNCTION falcon_7b_instruct_llm_model(super)
RETURNS super
SAGEMAKER '<endpointname>'
IAM_ROLE default;

Load sample amazon reviews data set into your Amazon Redshift data warehouse

In [0]:
CREATE TABLE sample_reviews
(
    review varchar(4000)
);

COPY sample_reviews 
FROM 's3://<<your bucket>>/sample_reviews.csv'
IAM_ROLE DEFAULT
csv
DELIMITER ','
IGNOREHEADER 1;

Create a UDF that engineers the prompt for sentiment analysis

In [0]:
CREATE FUNCTION udf_prompt_eng_sentiment_analysis (varchar)
  returns super
stable
as $$
  select json_parse(
  '{"inputs":"Classify the sentiment of this sentence as Positive, Negative, Neutral. Return only the sentiment nothing else.' || $1 || '","parameters":{"max_new_tokens":1000}}')
$$ language sql;

Make a remote inference to the LLM model to generate sentiment analysis for input dataset

In [0]:
CREATE table sentiment_analysis_for_reviews
as
(
    SELECT 
        review, 
        falcon_7b_instruct_llm_model
            (
                udf_prompt_eng_sentiment_analysis(review)
        ) as sentiment
    from sample_reviews
);

Analyze the output

In [0]:
SELECT review, sentiment[0]."generated_text" :: varchar as sentiment
FROM sentiment_analysis_for_reviews;